In [134]:
import numpy as np
import csv
import pickle
import pandas as pd


In [98]:
class myQuote:
    def __init__(self, text):
        #read in CSV, process
        self.quoteText = text
        self.quoteID = hash(self.quoteText)
        self.link = []
        self.name = []
        self.description = []
        self.scores = []
        self.cos = []
    
    def __hash__(self):
        return self.quoteID
    
    def __str__(self):
        return "Object text: " + self.quoteText + '\n' +\
        "Links: " + str(self.link) + '\n' +\
        "Names: " + str(self.name) + '\n' +\
        "Description " + str(self.description) + '\n' +\
        "Scores: " + str(self.scores)
        

In [229]:
infile = open('searchResults','rb')
results = pickle.load(infile, encoding='bytes')

In [230]:
#get the first 200 labels: those are the one in the mannual annotate sample
count = 0
proto_matrix = []
for item in results:
    #results[item].scores[0]
    row = results[item].scores
    row = np.append(row, results[item].cos)
    proto_matrix.append(row)
    count += 1
    if count > 199:
        break

matrix = np.matrix(proto_matrix)

In [ ]:
#read labels from ma

In [233]:
labels = pd.read_csv('MLData_recode_mannual.csv')
label = labels['label'].values
label = label.astype(np.float)
print(len(label))
print(type(label))

200
<class 'numpy.ndarray'>


In [323]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier

In [235]:
#split train test
X = matrix
y = label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [317]:
#####grid search (the parameters predict everything to one class, we should use a separated 
#sample for tuning parameters, but not enough cases so far)
parameters = [{'kernel': ['rbf','linear'], 'gamma': [0.01, 0.001, 0.0001],
                     'C': [1, 10, 100, 1000]}]
                   
grid_search_item = GridSearchCV(estimator = clf,
                          param_grid = parameters,
                           scoring = 'accuracy',
                           n_jobs = -1)
grid_search = grid_search_item.fit(X_train, y_train)

print(grid_search.best_score_)####not sure how come this is so good,but the parameters don't work good in the model
print(grid_search.best_params_)

0.8955223880597015
{'C': 100, 'gamma': 0.01, 'kernel': 'linear'}


In [320]:
clf = svm.SVC(gamma=0.01)
clf.fit(X_train, y_train) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [325]:
y = clf.predict(X_test)
cm = confusion_matrix(y_test, y)
print(cm)
print(f1_score(y_test, y, average= None) )

[[52  4]
 [ 7  3]]
[0.90434783 0.35294118]


####SGDClassifier 

In [326]:
#####grid search (the parameters predict everything to one class, we should use a separated 
#sample for tuning parameters, but not enough cases so far)
svc = svm.SVC(gamma="scale")
parameters = [{ 'alpha': [0.01, 0.05, 0.001, 0.005]}]
                   
grid_search_item = GridSearchCV(estimator = clf,
                          param_grid = parameters,
                           scoring = 'accuracy',
                           n_jobs = -1)
grid_search = grid_search_item.fit(X_train, y_train)

print(grid_search.best_score_)####not sure how come this is so good,but the parameters don't work good in the model
print(grid_search.best_params_)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Users/lucia/anaconda3/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/Users/lucia/anaconda3/lib/py

In [308]:

clf = SGDClassifier(n_iter=100, alpha=0.05)
clf.fit(X_train, y_train) 

/Users/lucia/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


SGDClassifier(alpha=0.05, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=None, n_iter=100,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=None, verbose=0, warm_start=False)

In [309]:
y = clf.predict(X_test)
y

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.])

In [305]:
y_test

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.])

In [310]:
f1_score(y_test, y, average= None) 

array([0.89473684, 0.33333333])

In [ ]:
########recode data

In [122]:
file  = pd.read_csv('MLData.csv')
file['label'][1:5]

1       quote
2       quote
3       quote
4    NotQuote
Name: label, dtype: object

In [125]:
def recode(series):
    if 'quote' in series:
        return 1
    else:
        return 0
    
file['label'] = file['label'].apply(recode)

In [127]:
print(file['label'][1:5])
file.to_csv('MLData_recode.csv')

1    1
2    1
3    1
4    0
Name: label, dtype: int64
